2023/12
## skku DataMining project : 
## 서울 대중교통 혼잡도 및 중심성 그래프분석(Seoul_transportation_GraphMining)
contributer : 김민, 김효원, 이수정, 조병웅  
instruction :   
블로거 GD park manger님의 대중교통 분석 연구를 바탕으로, 서울시 대중교통 네트워크의 중심성을 살펴보고,
사용목적 대비 유동인구 수라는 혼잡도 개념을 추가하여 새로 발전된 인사이트를 제공함을 목적으로 함.
참고한 분석 연구의 주소 -> https://m.blog.naver.com/gdpresent/221147909255

## find_centrality 파일 정보
좌표 데이터를 바탕으로 노드의 지역 정보를 추가하고, 해당 그래프를 바탕으로 centrality를 구하는 기본 코드.

## 라이브러리 및 데이터 로딩

In [37]:
#불러오기
import pandas as pd
import networkx as nx
node = pd.read_csv('../데이터/전처리2_생성데이터/혼잡지수_node.csv', encoding='cp949')
edges = pd.read_csv('../데이터/전처리2_생성데이터/혼잡지수_edge.csv', encoding='cp949')

In [38]:
#체크 부분
node

,title,ARS_ID,name,X,Y,type
0,1_4.19민주묘지역,1,4.19민주묘지역,37.649527,127.013689,subway
1,2_가능역,2,가능역,37.748393,127.044275,subway
2,3_가락시장역,3,가락시장역,37.493092,127.118258,subway
3,4_가산디지털단지역,4,가산디지털단지역,37.480389,126.882644,subway
4,5_가양역,5,가양역,37.561429,126.854402,subway
...,...,...,...,...,...,...
3257,68233_장냇말,68233,장냇말,37.726196,126.976828,bus
3258,68234_장골부대앞,68234,장골부대앞,37.731134,126.977772,bus
3259,68235_부곡1리마을회관,68235,부곡1리마을회관,37.736108,126.977397,bus
3260,68236_송추초등학교.송추검문소,68236,송추초등학교.송추검문소,37.716477,126.971601,bus


## 노드 정리
- 구 및 행정동 추가

In [39]:
#카카오 api
#카카오 디벨로퍼 사용법 -> https://m.blog.naver.com/imagine0716/220937236192

import requests
import json

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type" : "authorization_code",
    "client_id" : #개발자 본인 아이디 넣기
    "redirect_uri" : "https://localhost.com",
    "code"         : #api 코드 넣기

}
response = requests.post(url, data=data)

tokens = response.json()

In [40]:
#api 호출
!pip install PyKakao --upgrade
from PyKakao import Local

with open("kakao_token.json", "w") as fp:
    json.dump(tokens, fp)

api = Local(service_key = )#개발자 본인의 서비스키

In [41]:
#노드 컬럼 초기화
node['행정동'] = 0
node['구'] = 0

In [42]:
#체크 부분
node

,title,ARS_ID,name,X,Y,type,행정동,구
0,1_4.19민주묘지역,1,4.19민주묘지역,37.649527,127.013689,subway,0,0
1,2_가능역,2,가능역,37.748393,127.044275,subway,0,0
2,3_가락시장역,3,가락시장역,37.493092,127.118258,subway,0,0
3,4_가산디지털단지역,4,가산디지털단지역,37.480389,126.882644,subway,0,0
4,5_가양역,5,가양역,37.561429,126.854402,subway,0,0
...,...,...,...,...,...,...,...,...
3257,68233_장냇말,68233,장냇말,37.726196,126.976828,bus,0,0
3258,68234_장골부대앞,68234,장골부대앞,37.731134,126.977772,bus,0,0
3259,68235_부곡1리마을회관,68235,부곡1리마을회관,37.736108,126.977397,bus,0,0
3260,68236_송추초등학교.송추검문소,68236,송추초등학교.송추검문소,37.716477,126.971601,bus,0,0


In [43]:
#노드 컬럼에 실제 행정동 및 구 삽입
for i in node.index:
  y = node.loc[i]["X"]
  x = node.loc[i]["Y"]
  df = api.geo_coord2regioncode(x, y, dataframe=True)
  node.at[i,"행정동"] = df['region_3depth_name'][0]
  if len(df['region_2depth_name'][0].split()) == 2:
    node.at[i,"구"] = df['region_2depth_name'][0].split()[1]
  else:
    node.at[i,"구"] = df['region_2depth_name'][0]

In [44]:
#체크 부분
node

,title,ARS_ID,name,X,Y,type,행정동,구
0,1_4.19민주묘지역,1,4.19민주묘지역,37.649527,127.013689,subway,우이동,강북구
1,2_가능역,2,가능역,37.748393,127.044275,subway,가능동,의정부시
2,3_가락시장역,3,가락시장역,37.493092,127.118258,subway,가락동,송파구
3,4_가산디지털단지역,4,가산디지털단지역,37.480389,126.882644,subway,가산동,금천구
4,5_가양역,5,가양역,37.561429,126.854402,subway,등촌동,강서구
...,...,...,...,...,...,...,...,...
3257,68233_장냇말,68233,장냇말,37.726196,126.976828,bus,장흥면,양주시
3258,68234_장골부대앞,68234,장골부대앞,37.731134,126.977772,bus,장흥면,양주시
3259,68235_부곡1리마을회관,68235,부곡1리마을회관,37.736108,126.977397,bus,장흥면,양주시
3260,68236_송추초등학교.송추검문소,68236,송추초등학교.송추검문소,37.716477,126.971601,bus,장흥면,양주시


In [46]:
#저장용
import copy
node_copy = copy.deepcopy(node)


## 빠진 엣지 리스트 추가 및 그래프 완성

In [47]:
#엣지 리스트 추가하는 부분(수정님 코드)
edge_list = edges.loc[:, ('source', 'target')].values.tolist()
edges_weight=[]
for i in range(len(edge_list)):
    #edges_weight.append([edges.source[i],edges.target[i],{"weight":float(edges.total_support[i])}])
    edges_weight.append((edges.source[i],edges.target[i],edges.weight[i]))

#-----valid-----#
edges_weight

[('90_덕정역', '89_덕계역', 0.0027005517826825),
 ('63_남영역', '272_용산역', 0.072167869269949),
 ('272_용산역', '68_노량진역', 0.0535462648556876),
 ('68_노량진역', '84_대방역', 0.024140492359932),
 ('84_대방역', '210_신길역', 0.0398758488964346),
 ('210_신길역', '260_영등포역', 0.0592816213921901),
 ('260_영등포역', '217_신도림역', 0.1328162139219015),
 ('217_신도림역', '40_구로역', 0.1452408743633276),
 ('40_구로역', '45_구일역', 0.0827228353140916),
 ('45_구일역', '16_개봉역', 0.0792720713073005),
 ('262_오류동역', '265_온수역', 0.0608828522920203),
 ('265_온수역', '253_역곡역', 0.0500541171477079),
 ('181_서울역', '63_남영역', 0.0728013582342954),
 ('253_역곡역', '193_소사역', 0.0350891341256366),
 ('150_부천역', '310_중동역', 0.0153087860780984),
 ('310_중동역', '196_송내역', 0.0113338285229202),
 ('40_구로역', '4_가산디지털단지역', 0.0603809422750424),
 ('4_가산디지털단지역', '97_독산역', 0.05),
 ('97_독산역', '52_금천구청역', 0.0401994906621392),
 ('52_금천구청역', '184_석수역', 0.0344662563667232),
 ('184_석수역', '32_관악역', 0.0310918930390492),
 ('32_관악역', '236_안양역', 0.0282735568760611),
 ('236_안양역', '126_명학역', 0.021

In [48]:
edge = pd.read_csv('./혼잡지수_edge.csv', encoding='cp949')

In [49]:
#그래프 만들기
graph = nx.DiGraph()
graph.add_nodes_from(node.set_index('title').T.to_dict().items())
graph.add_edges_from(edge[['source', 'target']].values.tolist())
print(graph.number_of_nodes(), graph.number_of_edges())

3262 4642


In [50]:
#그래프에 빠진 엣지 리스트 추가하는 부분(수정님 코드)
graph.add_weighted_edges_from(edges_weight)

In [51]:
graph.edges(data=True)

OutEdgeDataView([('1_4.19민주묘지역', '194_솔밭공원역', {'weight': 0.0266369047619047}), ('1_4.19민주묘지역', '6_가오리역', {'weight': 0.0687587535014005}), ('1_4.19민주묘지역', '9120_인수동장미원', {'weight': 0.0078804690455175}), ('1_4.19민주묘지역', '10221_덕성여대입구', {'weight': 0.0519999999999978}), ('1_4.19민주묘지역', '9125_북부성모병원', {'weight': 0.0497777777777771}), ('2_가능역', '284_의정부역', {'weight': 0.0162913837011884}), ('2_가능역', '72_녹양역', {'weight': 0.011365662139219}), ('2_가능역', '61120_경기북부보훈지청', {'weight': 0.0109649122807016}), ('2_가능역', '61014_가재울교차로.일신건영휴먼빌아', {'weight': 0.0359649122806996}), ('3_가락시장역', '24_경찰병원역', {'weight': 0.0493120393120393}), ('3_가락시장역', '132_문정역', {'weight': 0.0900042808219178}), ('3_가락시장역', '203_수서역', {'weight': 0.0871483415233415}), ('3_가락시장역', '199_송파역', {'weight': 0.1619755993150685}), ('4_가산디지털단지역', '97_독산역', {'weight': 0.05}), ('4_가산디지털단지역', '321_철산역', {'weight': 0.0930363390585241}), ('4_가산디지털단지역', '40_구로역', {'weight': 0.0629679541595925}), ('4_가산디지털단지역', '60_남구로역', {'weight': 0.09705192

## 노드에 centrality 추가 및 상위 5개 보기

In [52]:
#노드 연결중심성(dc) 어트리뷰트에 어트리뷰트값 추가.
#의의 : 로컬리티를 굳이 고려하지 않아도 됨. / 엣지의 개수이기 때문에, 엣지가 많으면 -> 교통이 편리함
#엣지가 적으면 -> 교통이 불편함으로 인식 가능 / 노선이 적고, 아마 연결된 노드가 별로 없을 것이기 때문
#왜 엣지로 보냐 -> 대부분의 노선 정류장이 앞뒤로 노드를 한개만 갖기 때문에 노드 비교가 크게 의미가 없다.
dc = nx.degree_centrality(graph)
for i in list(graph.nodes()):
  graph.nodes[i]['dc'] = dc[i]

In [53]:
#상위 몇개 값 보여줌
sorted_nodes = sorted(dc.items(), key=lambda x:x[1], reverse=True)
print(sorted_nodes[:5])

#straightness centrality도 고려해볼만함.

[('181_서울역', 0.008892977614228765), ('154_불광역', 0.00827966881324747), ('256_연신내역', 0.0067463968107942356), ('28_공덕역', 0.006133088009812941), ('46_구파발역', 0.005519779208831647)]


In [54]:
#노드의 근접중심성
cc = nx.closeness_centrality(graph)
for i in list(graph.nodes()):
  graph.nodes[i]['cc'] = cc[i]
#상위 몇개 값 보여줌
cc_sorted_nodes = sorted(cc.items(), key=lambda x:x[1], reverse=True)
print(cc_sorted_nodes[:5])

[('272_용산역', 0.062339257062718735), ('181_서울역', 0.061869296951935916), ('287_이촌역', 0.061594450881773004), ('28_공덕역', 0.061460809187724404), ('63_남영역', 0.06131347261121425)]


In [55]:
#노드의 betweenness centrality
bc = nx.betweenness_centrality(graph)
for i in list(graph.nodes()):
  graph.nodes[i]['bc'] = bc[i]
#상위 몇개 값 보여줌
bc_sorted_nodes = sorted(bc.items(), key=lambda x:x[1], reverse=True)
print(bc_sorted_nodes[:5])

[('217_신도림역', 0.18983175975042196), ('181_서울역', 0.16752974169502843), ('272_용산역', 0.15615647688367768), ('287_이촌역', 0.1437397573289119), ('40_구로역', 0.14367374337051184)]


In [56]:
#노드의 weigted degree centrality
#원래 이 코드가 맞으나, 뭔가 문제가 있어 돌아가지 않음 아래로 대체
"""
wdc = nx.degree_centrality(graph, weight = 'weight')

for i in list(graph.nodes()):
  graph.nodes[i]['wdc'] = wdc[i]
#상위 몇개 값 보여줌
wdc_sorted_nodes = sorted(wdc.items(), key=lambda x:x[1], reverse=True)
print(wdc_sorted_nodes[:5])
"""

"\nwdc = nx.degree_centrality(graph, weight = 'weight')\n\nfor i in list(graph.nodes()):\n  graph.nodes[i]['wdc'] = wdc[i]\n#상위 몇개 값 보여줌\nwdc_sorted_nodes = sorted(wdc.items(), key=lambda x:x[1], reverse=True)\nprint(wdc_sorted_nodes[:5])\n"

In [58]:
#노드의 weigted degree centrality
import heapq

weighted_degree_centrality = {}
for node_one in graph.nodes():
    weighted_degree = sum(graph[node_one][neighbor]['weight'] for neighbor in graph.neighbors(node_one))
    weighted_degree_centrality[node_one] = weighted_degree
# Weighted Degree Centrality로 노드를 내림차순 정렬
sorted_node_ones = heapq.nlargest(5, weighted_degree_centrality, key=weighted_degree_centrality.get)

# 상위 5개 노드 출력
for node_one in sorted_node_ones:
    print(f"노드 {node_one}의 Weighted Degree Centrality: {weighted_degree_centrality[node_one]}")

노드 181_서울역의 Weighted Degree Centrality: 2.6779443893292303
노드 346_홍대입구역의 Weighted Degree Centrality: 2.2611563630608087
노드 28_공덕역의 Weighted Degree Centrality: 2.1426984285486097
노드 287_이촌역의 Weighted Degree Centrality: 1.9790755888271432
노드 107_디지털미디어시티역의 Weighted Degree Centrality: 1.8649256241226366


## 파일 저장

In [59]:
# 파일 저장
node.to_csv('./final_node.csv', index=None, encoding='cp949')
edge.to_csv('./final_edge.csv', index=None, encoding='cp949')
nx.write_graphml_lxml(graph, 'final_graph.graphml')

## 구 및 행정동 단위 서칭 코드 예시

In [65]:
#구 하나 보기
#변수 이름명은 구 영문명이나 아님 편하신대로 바꿔가시면서 보시면 됩니다. for 문으로 돌면서 보셔도 가능합니다.
gwangjingu = [n for n, attr in graph.nodes(data=True) if attr['구'] == '광진구']
gwangjingu_sub = graph.subgraph(gwangjingu)
sub_bc = nx.betweenness_centrality(gwangjingu_sub)
average_bc = sum(sub_bc.values()) / len(sub_bc) #bc, cc 등을 바꿔가시면서 보시면 됩니다.

In [66]:
average_bc

0.026251390433815348

In [63]:
#행정동 하나 보기
garakdong = [n for n, attr in graph.nodes(data=True) if attr['행정동'] == '가락동']
garakdong_sub = graph.subgraph(garakdong)
sub_bc = nx.betweenness_centrality(garakdong_sub)
average_bc = sum(sub_bc.values()) / len(sub_bc) #bc, cc 등을 바꿔가시면서 보시면 됩니다.

In [64]:
average_bc

0.3333333333333333